In [1]:
# Creates a parquet file with trajectories from raw AIS data

from make_trajectories_from_AIS import ais_to_trajectory

filename = '../../data/raw/AIS_04-07_2022_tromso/ais_202205.csv'  # read file from
size = -1  # set the number of AIS messages for processing (-1 processes the entire file)
start = 0  # index of the first message to process
save_to = '../../data/processed/202205_points_tromso_cleaned_meta_full_dualSplit_3.parquet'  # save file to

dates = ['202204', '202205', '202206', '202207']
for date in dates:
    filename = '../../data/raw/AIS_04-07_2022_tromso/ais_'+date+'.csv'
    save_to = '../../data/processed/'+date+'_points_tromso_cleaned_meta_full_dualSplit_3.parquet'
    ais_to_trajectory(filename, size, start, save_to)

Geopandas has version 0.13.2
Movingpandas has version 0.17.1
3207069 raw AIS messages loaded from file ../../data/raw/AIS_04-07_2022_tromso/ais_202204.csv
Ship metadata has   4142 unique MMSIs
AIS raw data has    246 unique MMSIs
Overlap:            228 MMSIs
2236248 superfluous AIS messages dropped, thereof
   226292 messages with irrelevant nav_status and
   2009956 duplicate messages.
Observation Gap splitter split 243 trajectories into 1396 sub-trajectories
Stop splitter split 1396 trajectories into 1956 sub-trajectories
0 trajectories were found that exceed the speed limit and dropped from the list of trajectories
Cleaning reduced 3207069 AIS messages to 771730 points (24.06%)
2728316 raw AIS messages loaded from file ../../data/raw/AIS_04-07_2022_tromso/ais_202205.csv
Ship metadata has   4142 unique MMSIs
AIS raw data has    228 unique MMSIs
Overlap:            204 MMSIs
1902265 superfluous AIS messages dropped, thereof
   151553 messages with irrelevant nav_status and
   1750712